<a href="https://colab.research.google.com/github/akanison12/Sudo-code-2024/blob/main/240114_NLPInVietNamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Library

In [ ]:
!pip install kaggle

! pip install vietnam-number
!kaggle datasets download -d tuannguyenvananh/vietnamese-text-classification-dataset
!unzip vietnamese-text-classification-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/tuannguyenvananh/vietnamese-text-classification-dataset
License(s): unknown
  0% 0.00/173k [00:00<?, ?B/s]
100% 173k/173k [00:00<00:00, 33.0MB/s]
Archive:  vietnamese-text-classification-dataset.zip
  inflating: train.csv               


In [ ]:
!apt-get update > /dev/null
!apt-get install -y openjdk-11-jdk > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!pip install -q py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from vietnam_number import n2w
import numpy as np
import pandas as pd
import py_vncorenlp
import contextlib
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from torch.optim import AdamW
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm



In [ ]:
!mkdir /content/vncorenlp
py_vncorenlp.download_model(save_dir='./vncorenlp/')

In [ ]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/vncorenlp')

#Read dataset

In [ ]:
file_path='/content/train.csv'

text_data = pd.read_csv(file_path, header=None, names=['label', 'content'])
text_data


,label,content
0,0,máy dùng hay bị đơ máy
1,0,chỉ có dây cáp nguồn không có adapter sao sử d...
2,0,Chất lượng quá kém Mới dùng được 2 ngày loa ba...
3,0,Usb tôi vừa mới nhận usb này Rất bực bội vì cá...
4,2,Tuyệt vời. Hàng FPT cửa hàng
...,...,...
3035,0,không có bộ thu đi kèm Vừa nhận hàng xong tức ...
3036,0,Cũng bình thường Không mạnh như kỹ vọng Chắc t...
3037,2,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...
3038,0,giao hàng lâu rửa tay một tí đã cháy màn hình ...


#Check Imbalanced data

In [ ]:
text_data['label'].value_counts()

,count
label,
0,1105
2,1048
1,887


In [ ]:
text_data.isnull().sum()

,0
label,0
content,0


#Check Duplicated

In [ ]:
duplicates_data = text_data[text_data.duplicated(subset=['content'], keep=False)].sort_values(by='content')
duplicates_data[['content', 'label']]

,content,label
2511,Bình thường Cách đây 2 hôm mình có đặt mua thì...,1
820,Bình thường Cách đây 2 hôm mình có đặt mua thì...,1
1086,Bình thường Giao hàng nhanh sản phẩm tốt tùy n...,1
727,Bình thường Giao hàng nhanh sản phẩm tốt tùy n...,1
1749,Bình thường Sản phẩm ok Nhưng hàng giao quá ch...,1
...,...,...
1608,ĐẶT MÀU CAM GIAO MÀU ĐEN Nhưng chất lượng máy ...,1
2695,đo sức khỏe không đúng nếu mua vì giá và không...,1
2002,đo sức khỏe không đúng nếu mua vì giá và không...,1
2421,Ổn nhưng chưa hài lòng lắm Mình mua sản phẩm c...,1


In [ ]:
text_data.drop_duplicates(subset=['content'], inplace=True)

In [ ]:
text_data.reset_index(drop=True, inplace=True)

In [ ]:
text_data['label'].value_counts()

,count
label,
0,1090
2,1041
1,867


#Prepare for Model

In [ ]:
import re

class Preprocessing:
    def __init__(self, n2w, rdrsegmenter):
        self.n2w = n2w
        self.rdrsegmenter = rdrsegmenter



    def processing(self, text):
        """Process input text by normalizing, replacing tokens, and segmenting."""
        # Step 1: Basic text cleaning and lowercase conversion
        text = self._clean_text(text)


        # Step 2: Replace emojis
        text = self._replace_emojis(text)

        # Step 3: Apply custom replacements for units and symbols
        text = self._replace_units_and_symbols(text)

        # Step 4: Convert numbers to words if they exist
        text = self._convert_numbers_to_words(text)

        #Step 5:Replace special character
        text = self._remove_special_characters(text)




        # Step 6: Text segmentation and normalization
        text = self._segment_text(text)

        return text

    def _clean_text(self, text):
        """Convert to lowercase and strip excess whitespace."""
        return text.strip().lower()

    def _replace_emojis(self, text):
        """Remove emojis from the text."""
        emoji_pattern = re.compile(
            "[" u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F700-\U0001F77F"  # alchemical symbols
            u"\U0001F780-\U0001F7FF"  # geometric shapes extended
            u"\U0001F800-\U0001F8FF"  # supplemental arrows-C
            u"\U0001F900-\U0001F9FF"  # supplemental symbols & pictographs
            u"\U0001FA00-\U0001FA6F"  # chess symbols
            u"\U0001FA70-\U0001FAFF"  # symbols & pictographs extended-A
            u"\U00002702-\U000027B0"  # dingbats
            u"\U000024C2-\U0001F251"  # enclosed characters
            "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    def _replace_units_and_symbols(self, text):
        """Replace shorthand units with their full word equivalents."""
        replacements = [
            (r'(\d+)k', r'\1 ngàn'),  # Replace 'k' with 'ngàn'
            (r'(\d+)%', r'\1 phần trăm'),  # Replace '%' with 'phần trăm'
            (r'(\d+)m(?![a-zA-Z])', r'\1 mét'),  # Replace 'm' with 'mét'
            (r'(\d+)s(?![a-zA-Z])', r'\1 giây'),  # Replace 's' with 'giây'
            (r'(\d+)\'(?![a-zA-Z])', r'\1 phút'),  # Replace '\'' with 'phút'
            (r'(\d+)h(?![a-zA-Z])', r'\1 giờ')  # Replace 'h' with 'giờ'
        ]
        for pattern, replacement in replacements:
            text = re.sub(pattern, replacement, text)
        return text

    def _convert_numbers_to_words(self, text):
        """Convert numeric values in text to their word representation."""
        return re.sub(r'\d+', lambda x: self.n2w(x.group()), text)
    def _remove_special_characters(self, text):
        """Remove special characters from the text."""
        return  re.sub(r'[^\w\s]', '', text)






    def _segment_text(self, text):
        """Normalize and segment the text."""
        return ''.join(self.rdrsegmenter.word_segment(text))


In [ ]:
examples = [
    "Hello 2k world! 😊",
    "Đi bộ 100m trong 5 phút",
    "Nhiệt độ giảm 10% trong 30s",
    "Chạy 1h với tốc độ 5km/h",
    "1234",
]
preprocessor = Preprocessing(n2w, rdrsegmenter)



for example in examples:
    print("Input:", example)
    print("Output:", preprocessor.processing(example))
    print("=" * 50)

Input: Hello 2k world! 😊
Output: hello hai ngàn world
Input: Đi bộ 100m trong 5 phút
Output: đi bộ một trăm mét trong năm phút
Input: Nhiệt độ giảm 10% trong 30s
Output: nhiệt_độ giảm mười phần_trăm trong ba_mươi giây
Input: Chạy 1h với tốc độ 5km/h
Output: chạy một giờ với tốc_độ năm ngànmh
Input: 1234
Output: một_nghìn_hai_trăm ba_mươi bốn


In [ ]:
data = text_data.copy()
data['content'] = data['content'].apply(preprocessor.processing)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    2998 non-null   int64 
 1   content  2998 non-null   object
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,data,tokenize,max_length=128):
        self.data = data
        self.text=data['content'].tolist()
        self.labels=data['label'].tolist()
        self.tokenizer=tokenize
        self.max_length=max_length

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):

        content=self.text[idx]
        label=self.labels[idx]
        encoding=self.tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }







In [ ]:
def collate_fn(batch):
  input_ids=[]
  attention_mask=[]
  label=[]
  for i  in batch:
    input_ids.append(i['input_ids'])
    attention_mask.append(i['attention_mask'])
    label.append(i['label'])

  input_ids= torch.nn.utils.rnn.pad_sequence(input_ids,
                                                    batch_first=True,
                                                    padding_value=tokenizer.pad_token_id)
  attention_mask= torch.nn.utils.rnn.pad_sequence(attention_mask,
                                                    batch_first=True,
                                                    padding_value=0)

  return input_ids,attention_mask,torch.tensor(label)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
dataset = CustomDataset(data,tokenizer)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42,shuffle=True,stratify=dataset.labels)
train_data, val_data = train_test_split(
    train_data,
    test_size=0.25,
    random_state=42,
    stratify=[item['label'] for item in train_data]
)


train_loader=DataLoader(train_data,batch_size=124,shuffle=True,collate_fn=collate_fn)
val_loader=DataLoader(val_data,batch_size=124,shuffle=False,collate_fn=collate_fn)
test_loader=DataLoader(test_data,batch_size=124,shuffle=False,collate_fn=collate_fn)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
phobert_model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)
optimizer = AdamW(phobert_model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
phobert_model.to(device)
num_epochs = 10


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Train and Test model

In [ ]:


def train(model, train_loader, val_loader, optimizer, device, num_epochs, patience=5):
    best_accuracy = 0
    no_improve = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training
        model.train()
        train_loss = 0
        train_accuracy = 0

        for batch in tqdm(train_loader, desc="Training"):
            input_ids, attention_mask, labels = batch

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            # Backpropagation
            loss = output.loss
            train_loss += loss.item()
            preds = torch.argmax(output.logits, dim=1)
            train_accuracy += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)
        avg_train_accuracy = train_accuracy / len(train_loader)

        print(f"Training loss: {avg_train_loss:.4f}")
        print(f"Training accuracy: {avg_train_accuracy:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids, attention_mask, labels = batch

                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

                val_loss += output.loss.item()
                preds = torch.argmax(output.logits, dim=1)
                val_accuracy += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = val_accuracy / len(val_loader)

        print(f"Validation loss: {avg_val_loss:.4f}")
        print(f"Validation accuracy: {avg_val_accuracy:.4f}")

        # Check for improvement
        if avg_val_accuracy > best_accuracy:
            best_accuracy = avg_val_accuracy
            no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved the best model.")
        else:
            no_improve += 1
            print(f"No improvement for {no_improve} epoch(s).")

        # Early stopping
        if no_improve >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}.")
            break

    print("Training complete.")













In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model_on_test(model, test_loader, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in test_loader:

            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Prediction
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)


            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    #  accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy:.4f}")

    # Classfication Report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))




In [ ]:
def inference(model,tokenizer,text):
  classes={0: "Tiêu cực ", 1: "Bình Thường",2: "Tích Cực"}
  model.eval()
  text=preprocessor.processing(text)
  encoding=tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  output=model(input_ids=encoding['input_ids'].to(device),attention_mask=encoding['attention_mask'].to(device))

  pre=torch.argmax(output.logits,dim=1)
  return classes[pre.item()]



#Train

In [ ]:
train(phobert_model, train_loader, val_loader, optimizer, device, num_epochs)



Epoch 1/10


Training: 100%|██████████| 15/15 [00:34<00:00,  2.29s/it]


Training loss: 1.0380
Training accuracy: 0.5086


Validation: 100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


Validation loss: 0.9068
Validation accuracy: 0.6685
Saved the best model.

Epoch 2/10


...

Validation loss: 0.4756
Validation accuracy: 0.8302
No improvement for 2 epoch(s).

Epoch 10/10


Training: 100%|██████████| 15/15 [00:34<00:00,  2.30s/it]


Training loss: 0.1742
Training accuracy: 0.9473


Validation: 100%|██████████| 5/5 [00:04<00:00,  1.03it/s]

Validation loss: 0.5124
Validation accuracy: 0.8399
No improvement for 3 epoch(s).
Training complete.


#Test

In [ ]:
ls

best_model.pth  models/  VnCoreNLP-1.2.jar


In [ ]:
best_model_path = 'best_model.pth'

# Load model
state_dict = torch.load(best_model_path)
phobert_model.load_state_dict(state_dict)

# Eval model
evaluate_model_on_test(phobert_model, test_loader, device)


<ipython-input-55-33f47c6a377a>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path)


Test Accuracy: 0.8233

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       218
           1       0.71      0.69      0.70       174
           2       0.91      0.87      0.89       208

    accuracy                           0.82       600
   macro avg       0.82      0.82      0.82       600
weighted avg       0.82      0.82      0.82       600



#Inference

#Level:Easy

In [ ]:
text="Món ăn này rất ngon, tôi thích cách trình bày của nó."
inference(phobert_model,tokenizer,text)



'Tích Cực'

In [ ]:
text="Món ăn này ổn, không có gì đặc biệt."
inference(phobert_model,tokenizer,text)

'Bình Thường'

In [ ]:
text="Món ăn này quá mặn và không hợp khẩu vị của tôi."
inference(phobert_model,tokenizer,text)

'Tiêu cực '

#Level:Medium

In [ ]:
text="Nhân viên rất thân thiện, dịch vụ ở đây khiến tôi cảm thấy hài lòng."
inference(phobert_model,tokenizer,text)

'Tích Cực'

In [ ]:
text="Nhân viên rất thân thiện, dịch vụ ở đây khiến tôi cảm thấy hài lòng."
inference(phobert_model,tokenizer,text)

'Tích Cực'

In [ ]:
text="Nhân viên phục vụ quá chậm, tôi phải đợi rất lâu."
inference(phobert_model,tokenizer,text)

'Tiêu cực '

#Level:Hard

In [ ]:
text="Ứng dụng này rất dễ sử dụng, các tính năng được thiết kế trực quan và thực sự giúp tôi tiết kiệm thời gian."

inference(phobert_model,tokenizer,text)

'Tích Cực'

In [ ]:
text="Ứng dụng này có đầy đủ chức năng cơ bản, nhưng không có gì nổi bật so với các ứng dụng khác."
inference(phobert_model,tokenizer,text)

'Bình Thường'

In [ ]:
text="Ứng dụng này liên tục bị lỗi, giao diện khó hiểu, và không thực sự hữu ích."
inference(phobert_model,tokenizer,text)

'Tiêu cực '